In [ ]:
!pip install plotly folium python-dotenv


In [ ]:
import pandas as pd

df = pd.read_csv("cleaned_weather.csv")
print("✅ File loaded — Shape:", df.shape)
df.head()


✅ File loaded — Shape: (8122, 41)


,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1715849100,2024-05-16 13:15,26.6,79.8,Partly Cloudy,...,8.4,26.6,1.0,1.0,04:50 AM,06:50 PM,12:12 PM,01:11 AM,Waxing Gibbous,55.0
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1715849100,2024-05-16 10:45,19.0,66.2,Partly cloudy,...,1.1,2.0,1.0,1.0,05:21 AM,07:54 PM,12:58 PM,02:14 AM,Waxing Gibbous,55.0
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1715849100,2024-05-16 09:45,23.0,73.4,Sunny,...,10.4,18.4,1.0,1.0,05:40 AM,07:50 PM,01:15 PM,02:14 AM,Waxing Gibbous,55.0
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1715849100,2024-05-16 10:45,6.3,43.3,Light drizzle,...,0.7,0.9,1.0,1.0,06:31 AM,09:11 PM,02:12 PM,03:31 AM,Waxing Gibbous,55.0
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1715849100,2024-05-16 09:45,26.0,78.8,Partly cloudy,...,183.4,262.3,5.0,10.0,06:12 AM,05:55 PM,01:17 PM,12:38 AM,Waxing Gibbous,55.0


DATA COLUMN & DETECTING KEY VARIABLES


In [ ]:
import numpy as np
import pandas as pd

# Convert to datetime if not already
if "year_month" in df.columns:
    df["year_month"] = pd.PeriodIndex(df["year_month"], freq="M").to_timestamp()
elif "date" in df.columns:
    df["year_month"] = pd.to_datetime(df["date"], errors="coerce").dt.to_period("M").dt.to_timestamp()
elif "last_updated" in df.columns:
    df["year_month"] = pd.to_datetime(df["last_updated"], errors="coerce").dt.to_period("M").dt.to_timestamp()
else:
    raise SystemExit("No 'year_month', 'date', or 'last_updated' column found.")

# Helper function to detect columns by keywords
def pick_col(candidates):
    for c in df.columns:
        cl = c.lower()
        if cl in candidates or any(k in cl for k in candidates):
            return c
    return None

temp_col = pick_col(["temperature","avgtemp_c","temp_c","temperature_c","maxtemp_c","feels_like_c"])
hum_col  = pick_col(["humidity","humid"])
prec_col = pick_col(["precip_mm","precipitation","rain_mm"])
wind_col = pick_col(["wind_kph","wind_speed","windspeed","wind_mph"])
country_col = pick_col(["country","location","name"])

print("Detected columns:")
print("Temperature:", temp_col)
print("Humidity:", hum_col)
print("Precipitation:", prec_col)
print("Wind:", wind_col)
print("Country:", country_col)

Detected columns:
Temperature: temperature_celsius
Humidity: humidity
Precipitation: precip_mm
Wind: wind_mph
Country: country


DESCRIPTIVE STATISTICS + CORRELATION MATRIX


In [ ]:
num = df.select_dtypes(include="number")
desc = num.describe().round(3)
corr = num.corr()

# Save reports
desc.to_csv("stats_descriptive.csv")
corr.to_csv("stats_correlation.csv")

print("✅ Descriptive & Correlation reports saved")
desc.head()


✅ Descriptive & Correlation reports saved


,latitude,longitude,last_updated_epoch,temperature_celsius,temperature_fahrenheit,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,...,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,moon_illumination
count,8122.000,8122.000,8.122000e+03,8122.000,8122.000,8122.000,8122.000,8122.000,8122.000,8122.000,...,8121.000,8121.000,8121.000,8121.000,8121.000,8121.000,8121.000,8121.000,8121.000,8121.000
mean,19.206,21.648,1.717544e+09,25.860,78.548,9.048,14.566,175.641,1012.433,29.896,...,22.262,515.299,62.111,9.582,6.731,17.378,32.050,1.374,1.818,60.074
std,24.527,65.665,1.038314e+06,7.193,12.947,21.156,34.049,100.266,5.781,0.171,...,34.813,1593.438,42.411,23.628,16.699,61.218,81.636,0.815,1.896,34.174
min,-41.300,-175.200,1.715849e+09,-3.700,25.300,2.200,3.600,1.000,978.000,28.890,...,3.600,-9999.000,0.000,0.000,0.000,0.500,0.500,1.000,1.000,0.000
25%,3.480,-6.840,1.716646e+09,21.825,71.325,4.300,6.800,91.000,1009.000,29.800,...,13.600,190.300,28.600,0.500,0.400,2.000,3.600,1.000,1.000,29.000


HEATMAP


In [ ]:
import plotly.express as px

fig_corr = px.imshow(corr, labels=dict(color="Correlation"), title="Correlation Heatmap")
fig_corr.show()


SEASONAL PATTERN ANALYSIS


In [ ]:
df["month"] = pd.to_datetime(df["year_month"]).dt.month

if temp_col:
    s_t = df.groupby("month")[temp_col].mean().reset_index()
    fig_temp = px.line(s_t, x="month", y=temp_col, markers=True, title="Seasonality: Average Temperature by Month")
    fig_temp.show()

if prec_col:
    s_p = df.groupby("month")[prec_col].mean().reset_index()
    fig_prec = px.line(s_p, x="month", y=prec_col, markers=True, title="Seasonality: Average Precipitation by Month")
    fig_prec.show()


GLOBAL TIME-SERIES TRENDS

In [ ]:
if temp_col:
    ts_t = df.groupby("year_month")[temp_col].mean().reset_index()
    px.line(ts_t, x="year_month", y=temp_col, title="Global Temperature Trend").show()

if prec_col:
    ts_p = df.groupby("year_month")[prec_col].mean().reset_index()
    px.line(ts_p, x="year_month", y=prec_col, title="Global Precipitation Trend").show()


REGIONAL COMPARISON & CHOROPLETH


In [ ]:
if country_col and temp_col:
    top = (df.groupby(country_col)[temp_col].count()
             .sort_values(ascending=False).head(10).index.tolist())
    df_top = df[df[country_col].isin(top)]
    px.line(df_top, x="year_month", y=temp_col, color=country_col,
            title=f"Top 10 Countries: {temp_col} over time").show()

    geo = (df.groupby(country_col)[temp_col].mean().reset_index()
             .rename(columns={temp_col: f"mean_{temp_col}"}))
    fig_ch = px.choropleth(
        geo,
        locations=country_col,
        locationmode="country names",
        color=f"mean_{temp_col}",
        title=f"Choropleth: Mean {temp_col} by Country"
    )
    fig_ch.show()


SCATTER PLOT(TEMP VS HUMITIDITY)

In [ ]:
if temp_col and hum_col:
    samp = df.sample(min(len(df), 10000), random_state=42)
    px.scatter(samp, x=temp_col, y=hum_col, opacity=0.5, title="Temperature vs Humidity").show()


EXTREME EVENT DETECTION

In [ ]:
ext_counts = {}
for label, col in {"temperature": temp_col, "precip": prec_col, "wind": wind_col}.items():
    if not col:
        continue
    g = df.dropna(subset=[col]).copy()
    if country_col:
        g = g.groupby(country_col, group_keys=False).apply(
            lambda x: x.assign(**{f"z_{col}": (x[col]-x[col].mean())/(x[col].std(ddof=0) or 1)})
        )
    else:
        g[f"z_{col}"] = (g[col]-g[col].mean())/(g[col].std(ddof=0) or 1)

    g[f"{col}_is_extreme"] = g[f"z_{col}"].abs() >= 2
    extremes = g[g[f"{col}_is_extreme"]]
    extremes.to_csv(f"extremes_{label}.csv", index=False)
    ext_counts[label] = len(extremes)

ext_counts


/tmp/ipython-input-2343236067.py:7: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/tmp/ipython-input-2343236067.py:7: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/tmp/ipython-input-2343236067.py:7: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass

{'temperature': 302, 'precip': 338, 'wind': 331}

In [ ]:
from google.colab import files
files.download("stats_descriptive.csv")
files.download("stats_correlation.csv")
files.download("extremes_temperature.csv")

# download others if generated
# files.download("extremes_precip.csv")
# files.download("extremes_wind.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>